In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2023-ml-w4p1/train.csv
/kaggle/input/2023-ml-w4p1/test.csv
/kaggle/input/2023-ml-w4p1/sample_submit.csv


In [2]:
%config Completer.use_jedi = False

In [3]:
import random
import os
seed = 42
random.seed(seed)
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
train = pd.read_csv("/kaggle/input/2023-ml-w4p1/train.csv")
test = pd.read_csv("/kaggle/input/2023-ml-w4p1/test.csv")
submit = pd.read_csv("/kaggle/input/2023-ml-w4p1/sample_submit.csv")

In [6]:
train

,N,P,K,temperature,humidity,ph,rainfall,label
0,22,36,16,30.581395,50.771481,8.184229,64.585596,mothbeans
1,18,27,41,22.365094,92.308824,7.175344,104.821633,pomegranate
2,61,68,50,35.214628,91.497251,6.793245,243.074507,papaya
3,26,72,22,28.767949,37.577921,4.674942,91.720849,pigeonpeas
4,2,140,197,22.697801,92.822234,5.534567,105.050823,apple
...,...,...,...,...,...,...,...,...
1645,10,5,5,21.213070,91.353492,7.817846,112.983436,orange
1646,108,94,47,27.359116,84.546250,6.387431,90.812505,banana
1647,11,36,31,27.920633,51.779659,6.475449,100.258567,mango
1648,11,124,204,13.429886,80.066340,6.361141,71.400430,grapes


In [7]:
X = train.drop('label', axis=1)
print(X.shape)

y = train['label']
print(y.shape)

(1650, 7)
(1650,)


In [8]:
# 혹시 모르니깐 아이리스 
import seaborn as sns
iris = sns.load_dataset('iris')
print(iris.head())

   sepal_length  sepal_width  petal_length  petal_width species
0           5.1          3.5           1.4          0.2  setosa
1           4.9          3.0           1.4          0.2  setosa
2           4.7          3.2           1.3          0.2  setosa
3           4.6          3.1           1.5          0.2  setosa
4           5.0          3.6           1.4          0.2  setosa


In [9]:
y

0         mothbeans
1       pomegranate
2            papaya
3        pigeonpeas
4             apple
           ...     
1645         orange
1646         banana
1647          mango
1648         grapes
1649         lentil
Name: label, Length: 1650, dtype: object

In [10]:
# 문자 라벨 숫자로 바꾸기
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
#yy = le.fit_transform(y['label'].values) 
#yy = yy.values.ravel() # labels.squeeze()#y가 데이터프레임일 경우
yy = le.fit_transform(y)
print(np.unique(yy))

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]


In [11]:
from sklearn.model_selection import train_test_split as sp

X_train, X_test, y_train, y_test = sp(X, y, random_state=1, test_size=0.2, stratify=y)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1320, 7)
(330, 7)
(1320,)
(330,)


In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

ss = StandardScaler()
ms = MinMaxScaler()

X_train_ss = ss.fit_transform(X_train)
X_test_ss = ss.transform(X_test)

X_train_ms = ms.fit_transform(X_train)
X_test_ms = ms.transform(X_test)

In [13]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

knn = KNeighborsClassifier()
logi = LogisticRegression()
# lda = LinearDiscriminantAnalysis()
# qda = QuadraticDiscriminantAnalysis()

In [14]:
from sklearn.model_selection import GridSearchCV

SCORING = 'accuracy'

param_knn = {"n_neighbors":[3, 4, 5, 6], "p":[1, 2], "weights":['uniform', 'distance']}
param_logi = {'solver':['saga'], "penalty":['ll', 'l2'], 'C':[0.1,1,5,10,100], 'max_iter':[5000]}
param_lda = {'solver':['lsqr', 'eigen'], 'shrinkage':['None', 'auto']}
param_qda = {'store_covariance':[True, False]}

gscv_knn = GridSearchCV(estimator = knn, param_grid = param_knn, scoring = SCORING, cv = 3, refit = True)
gscv_logi = GridSearchCV(estimator = logi, param_grid = param_logi, scoring = SCORING, cv = 3, refit = True)
gscv_lda = GridSearchCV(estimator = lda, param_grid = param_lda, scoring = SCORING, cv = 3, refit = True)
gscv_qda = GridSearchCV(estimator = qda, param_grid = param_qda, scoring = SCORING, cv = 3, refit = True)

In [15]:
#gscv_knn.fit(X_train, y_train)
#gscv_logi.fit(X_train, y_train)
#gscv_lda.fit(X_train, y_train)
#gscv_qda.fit(X_train, y_train)

In [16]:
# print(gscv_knn.best_params_)
# print(gscv_knn.best_score_)
# print(gscv_logi.best_params_)
# print(gscv_logi.best_score_)
# print(gscv_lda.best_params_)
# print(gscv_lda.best_score_)
# print(gscv_qda.best_params_)
# print(gscv_qda.best_score_)

In [17]:
model = QuadraticDiscriminantAnalysis(store_covariance = True)

In [18]:
model.fit(X, yy)

QuadraticDiscriminantAnalysis(store_covariance=True)

In [19]:
y_pred = model.predict(test)
y_pred

array([15, 21, 17, 17,  0, 12,  0, 13, 14, 10,  2,  4, 19,  8,  4, 19,  0,
       11, 17, 15,  5, 17, 16, 17,  3,  8, 14, 16, 18, 20, 19, 13,  8, 10,
        8,  2,  8,  3,  3,  9, 17, 12,  2, 11, 14, 11, 18,  4, 15, 11,  2,
        5,  7, 14,  5,  9,  6,  0,  1,  2, 21,  4, 10, 16, 17, 18, 16, 20,
       15, 18, 15,  4,  8,  1,  2, 17,  1,  6, 21, 16,  5,  3, 20, 13, 16,
       12,  5, 13,  2, 19, 11, 13,  6, 17, 18, 13,  9,  5,  2, 10,  4, 20,
       16, 15, 21,  9, 21,  1, 18, 13,  1,  8,  6, 19, 18,  3, 11,  4, 19,
       20, 18,  7,  2,  4,  3,  2,  4, 11,  1, 13,  1,  9, 19,  3,  4, 16,
       18,  1,  1,  0,  9, 15, 14, 13,  4, 11,  0,  4,  9, 13, 14, 10, 21,
       14, 18, 18, 18,  9, 11,  8,  3,  0, 16,  6, 20,  4,  7, 10, 21,  7,
        7,  2, 19,  3,  4, 11, 10,  7, 21,  8,  5,  5,  9,  8, 13,  9,  1,
        9,  4, 17, 17, 14, 12, 19, 21,  9, 11,  0,  2,  3,  7,  7,  1,  6,
       20, 19, 14,  1,  8, 14, 11,  3,  3,  3,  0, 20,  9, 17,  5,  2,  9,
       12, 12,  4, 17,  0

In [20]:
yyy = le.inverse_transform(y_pred)

In [21]:
yyy

array(['muskmelon', 'watermelon', 'papaya', 'papaya', 'apple', 'mango',
       'apple', 'mothbeans', 'mungbean', 'lentil', 'blackgram', 'coconut',
       'pomegranate', 'jute', 'coconut', 'pomegranate', 'apple', 'maize',
       'papaya', 'muskmelon', 'coffee', 'papaya', 'orange', 'papaya',
       'chickpea', 'jute', 'mungbean', 'orange', 'pigeonpeas', 'rice',
       'pomegranate', 'mothbeans', 'jute', 'lentil', 'jute', 'blackgram',
       'jute', 'chickpea', 'chickpea', 'kidneybeans', 'papaya', 'mango',
       'blackgram', 'maize', 'mungbean', 'maize', 'pigeonpeas', 'coconut',
       'muskmelon', 'maize', 'blackgram', 'coffee', 'grapes', 'mungbean',
       'coffee', 'kidneybeans', 'cotton', 'apple', 'banana', 'blackgram',
       'watermelon', 'coconut', 'lentil', 'orange', 'papaya',
       'pigeonpeas', 'orange', 'rice', 'muskmelon', 'pigeonpeas',
       'muskmelon', 'coconut', 'jute', 'banana', 'blackgram', 'papaya',
       'banana', 'cotton', 'watermelon', 'orange', 'coffee', 'chickp

In [22]:
submit['label'] = yyy
submit.to_csv("submit.csv", index=False)